In [20]:
import nltk; nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cbeams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [22]:
# Import Dataset
df = pd.read_json('../raw_data/all_data_df_stripped.json')
print(df.artist.unique())
df.head()

['Adam Devine' 'Adam Hess' 'Adam Hills' 'Adam Sandler' 'Adel Karam'
 'Aisling Bea' 'Al Madrigal' 'Al Murray' 'Al Porter' 'Alan Carr'
 'Alex Edelman' 'Ali Wong' 'Alistair McGowan' 'Amanda Seales'
 'Amy Schumer' 'Andi Osho' 'Andrew Lawrence' 'Andrew Maxwell'
 'Andy Parsons' 'Andy Woodhull' 'Angela Barnes' 'Anjelah Johnson'
 'Anthony Jeselnik' 'Ari Shaffir' 'Arj Barker' 'Arsenio Hall'
 'Aziz Ansari' 'Bert Kreischer' 'Big Jay Oakerson' 'Bill Burr'
 'Bill Hicks' 'Bill Maher' 'Billy Connolly' 'Bo Burnham' 'Bob Monkhouse'
 'Brad Williams' 'Brent Morin' 'Brian Regan' 'Bridget Everett'
 'Cedric The Entertainer' 'Celia Pacquola' 'Chelsea Peretti'
 'Chris Addison' "Chris D'Elia" 'Chris Gethard' 'Chris McCausland'
 'Chris Moyles' 'Chris Ramsey' 'Chris Rock' 'Chris Tucker'
 'Christina Pazsitzky' 'Colin Quinn' 'Craig Ferguson' 'Cristela Alonzo'
 'D.l. Hughley' 'Dan Soder' 'Dana Carvey' 'Dane Baptiste' 'Daniel Tosh'
 'Danny Bhoy' 'Dara Ó Briain' 'Darren Harriott' 'Dave Allen' 'Dave Attell'
 'Dave Cha

,full_transcript,artist,show_name,year,source,artist_birthday,artist_gender,age_then,full_transcript_clean
0,[rock music playing]\n\n[indistinct chatter]\n...,Adam Devine,Best Time Of Our Lives,2019.0,Scraps from the Loft,1983.0,2,36.0,man let right guys much take ...
1,"strong language. CHEERING Yes, yes, yes! How...",Adam Hess,Live from the BBC,2016.0,BBC,1990.0,2,26.0,yes yes yes well lovely m going apolog...
2,"Ladies and gentlemen, please welcome your hos...",Adam Hills,Live at the Apollo Series 9 Episode 4,2013.0,BBC,1970.0,2,43.0,please host tonight london london live ...
3,some strong language and adult humour Ladies...,Adam Hills,Live at the Apollo Series 12 Episode 4,2016.0,BBC,1970.0,2,46.0,please host tonight nt know hair ok lo...
4,"? CHEERING Hello, Apollo. I am going to start ...",Adam Hills,Live at the Apollo Series 5 Episode 5,2009.0,BBC,1970.0,2,39.0,going start saying something ve probably ne...


In [24]:
# Convert to list
data = df.full_transcript_clean.values.tolist()

pprint(data[:1])

[' man    let           right   guys much  take seat    oh   guys got jacked   '
 'fully juiced thing  man    excited  guys   gon na blast tonight    excited '
 'be  get excited  though    adult man  know   get excited  lock   get '
 'littlekidlevel   geeked  know  remember kid would   open present   '
 'vocabulary say    much  mama  however guys talked   know  weirdo  said '
 'things    mama  much  mama   vocabulary say  went like make face adult  lock '
 'ass     adultass things take kid school  drop    enjoy lunch today  made  '
 'ham cheese   questions quality meats  know  made   littlekidlevel excited '
 'face   months  child  swear  mom  uh  gave super bouncy ball birthday  guys  '
 'poor parents   cool  uh   gave bouncy ball  right  months straight  driveway '
 'like snatched midair   phrase use youth   midair  nothing midair adult  '
 'child   caught midair  mom  soaring abyss  snatched midair  midair  mom  '
 'midair  midair  midair  mom    jesus christ   course caught midair 

In [25]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['man', 'let', 'right', 'guys', 'much', 'take', 'seat', 'oh', 'guys', 'got', 'jacked', 'fully', 'juiced', 'thing', 'man', 'excited', 'guys', 'gon', 'na', 'blast', 'tonight', 'excited', 'be', 'get', 'excited', 'though', 'adult', 'man', 'know', 'get', 'excited', 'lock', 'get', 'littlekidlevel', 'geeked', 'know', 'remember', 'kid', 'would', 'open', 'present', 'vocabulary', 'say', 'much', 'mama', 'however', 'guys', 'talked', 'know', 'weirdo', 'said', 'things', 'mama', 'much', 'mama', 'vocabulary', 'say', 'went', 'like', 'make', 'face', 'adult', 'lock', 'ass', 'adultass', 'things', 'take', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'made', 'ham', 'cheese', 'questions', 'quality', 'meats', 'know', 'made', 'littlekidlevel', 'excited', 'face', 'months', 'child', 'swear', 'mom', 'uh', 'gave', 'super', 'bouncy', 'ball', 'birthday', 'guys', 'poor', 'parents', 'cool', 'uh', 'gave', 'bouncy', 'ball', 'right', 'months', 'straight', 'driveway', 'like', 'snatched', 'midair', 'phrase', 'use',

In [26]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['man', 'let', 'right', 'guys', 'much', 'take', 'seat', 'oh', 'guys', 'got', 'jacked', 'fully', 'juiced', 'thing', 'man', 'excited', 'guys', 'gon', 'na', 'blast', 'tonight', 'excited', 'be', 'get', 'excited', 'though', 'adult', 'man', 'know', 'get', 'excited', 'lock', 'get', 'littlekidlevel', 'geeked', 'know', 'remember', 'kid', 'would', 'open', 'present', 'vocabulary', 'say', 'much', 'mama', 'however', 'guys', 'talked', 'know', 'weirdo', 'said', 'things', 'mama', 'much', 'mama', 'vocabulary', 'say', 'went', 'like', 'make', 'face', 'adult', 'lock', 'ass', 'adultass', 'things', 'take', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'made', 'ham', 'cheese', 'questions', 'quality', 'meats', 'know', 'made', 'littlekidlevel', 'excited', 'face', 'months', 'child', 'swear', 'mom', 'uh', 'gave', 'super', 'bouncy', 'ball', 'birthday', 'guys', 'poor', 'parents', 'cool', 'uh', 'gave', 'bouncy', 'ball', 'right', 'months', 'straight', 'driveway', 'like', 'snatched', 'midair', 'phrase', 'use', 

In [27]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [29]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['man', 'let', 'right', 'guy', 'much', 'take', 'seat', 'guy', 'get', 'jack', 'fully', 'juiced', 'thing', 'man', 'excited', 'guy', 'going', 'blast', 'tonight', 'excited', 'get', 'excite', 'adult', 'man', 'know', 'get', 'excited', 'lock', 'get', 'littlekidlevel', 'geeke', 'know', 'remember', 'kid', 'open', 'present', 'vocabulary', 'say', 'much', 'mama', 'however', 'guy', 'talk', 'know', 'say', 'thing', 'much', 'say', 'go', 'make', 'face', 'adult', 'lock', 'ass', 'adultass', 'thing', 'take', 'kid', 'school', 'drop', 'enjoy', 'lunch', 'today', 'make', 'ham', 'cheese', 'question', 'quality', 'meat', 'know', 'make', 'littlekidlevel', 'excited', 'face', 'month', 'child', 'swear', 'mom', 'give', 'super', 'bouncy', 'ball', 'birthday', 'guy', 'poor', 'parent', 'cool', 'give', 'bouncy', 'ball', 'right', 'month', 'straight', 'driveway', 'snatch', 'midair', 'phrase', 'youth', 'adult', 'child', 'catch', 'mom', 'soar', 'abyss', 'snatch', 'midair', 'mom', 'mom', 'course', 'catch', 'midair', 'honey', 

In [30]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 9), (1, 2), (2, 1), (3, 1), (4, 1), (5, 4), (6, 1), (7, 1), (8, 6), (9, 1), (10, 3), (11, 1), (12, 10), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 5), (19, 3), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 15), (26, 3), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 3), (34, 8), (35, 3), (36, 1), (37, 4), (38, 4), (39, 1), (40, 4), (41, 2), (42, 1), (43, 8), (44, 1), (45, 9), (46, 3), (47, 1), (48, 7), (49, 1), (50, 1), (51, 5), (52, 3), (53, 5), (54, 1), (55, 1), (56, 4), (57, 1), (58, 3), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 8), (71, 2), (72, 2), (73, 2), (74, 1), (75, 2), (76, 1), (77, 7), (78, 3), (79, 13), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 2), (86, 2), (87, 7), (88, 1), (89, 1), (90, 1), (91, 1), (92, 5), (93, 7), (94, 2), (95, 1), (96, 5), (97, 1), (98, 1), (99, 11), (100, 1), (101, 2), (102, 1), (103, 2), (104, 1), (105, 2), (106, 1), (107, 1), (108, 1), (109, 1), (110

In [31]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [32]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.035*"go" + 0.029*"know" + 0.029*"get" + 0.014*"guy" + 0.014*"right" + '
  '0.012*"say" + 0.012*"want" + 0.011*"think" + 0.010*"look" + 0.010*"going"'),
 (1,
  '0.051*"get" + 0.033*"shit" + 0.027*"know" + 0.027*"say" + 0.023*"go" + '
  '0.022*"man" + 0.015*"going" + 0.015*"right" + 0.015*"black" + 0.015*"come"'),
 (2,
  '0.000*"get" + 0.000*"go" + 0.000*"know" + 0.000*"say" + 0.000*"right" + '
  '0.000*"man" + 0.000*"want" + 0.000*"come" + 0.000*"going" + 0.000*"see"'),
 (3,
  '0.000*"get" + 0.000*"know" + 0.000*"say" + 0.000*"go" + 0.000*"want" + '
  '0.000*"people" + 0.000*"see" + 0.000*"man" + 0.000*"come" + 0.000*"right"'),
 (4,
  '0.024*"know" + 0.022*"people" + 0.017*"say" + 0.014*"think" + 0.011*"get" + '
  '0.011*"thing" + 0.008*"make" + 0.008*"time" + 0.007*"want" + 0.007*"see"'),
 (5,
  '0.002*"decker" + 0.002*"archaeology" + 0.000*"garter" + 0.000*"get" + '
  '0.000*"know" + 0.000*"go" + 0.000*"likelihood" + 0.000*"right" + '
  '0.000*"people" + 0.000*"say"'),
 (6,


In [33]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.17708754040126

Coherence Score:  0.25788326013983226


In [36]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

/Users/cbeams/.pyenv/versions/3.8.12/envs/objectively_funny/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.392613 -0.128759       1        1  44.913184
4     -0.395404 -0.043280       2        1  19.541319
9     -0.373864 -0.173631       3        1  16.774905
1     -0.347228 -0.288416       4        1  12.949969
12    -0.418670  0.106500       5        1   5.709700
19     0.128112  0.116564       6        1   0.068830
8      0.105122  0.073829       7        1   0.035553
5      0.130535  0.024916       8        1   0.001355
13     0.130741  0.025518       9        1   0.000858
7      0.130757  0.026093      10        1   0.000820
11     0.130251  0.026067      11        1   0.000373
14     0.130251  0.026067      12        1   0.000357
16     0.130251  0.026067      13        1   0.000357
18     0.130251  0.026067      14        1   0.000356
2      0.130251  0.026067      15        1   0.000356
6      0.130251  0.026067      16        1   0.000354
3      0.130251  0.026067      17        1   0.000341
17     0.130251  0.026067      18        1   0.000338
10     0.130251  0.026067      19        1   0.000338
15     0.130251  0.026067      20        1   0.000338, topic_info=       Term          Freq         Total Category  logprob  loglift
342     get  35288.000000  35288.000000  Default  30.0000  30.0000
349      go  36410.000000  36410.000000  Default  29.0000  29.0000
729    shit   8184.000000   8184.000000  Default  28.0000  28.0000
702     say  22789.000000  22789.000000  Default  27.0000  27.0000
691   right  13433.000000  13433.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
676  really      0.000119   6422.045705  Topic20 -10.4658  -5.2023
729    shit      0.000119   8184.213329  Topic20 -10.4658  -5.4448
802    take      0.000119   7234.104107  Topic20 -10.4658  -5.3214
812    tell      0.000119   6824.299411  Topic20 -10.4658  -5.2631
829    time      0.000119   8856.482371  Topic20 -10.4658  -5.5237

[1392 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5165      1  0.170791   abortion
5165      2  0.825489   abortion
2140      1  0.304807     accent
2140      3  0.049162     accent
2140      4  0.039330     accent
...     ...       ...        ...
1421      2  0.002580       yell
1421      4  0.012899       yell
4717      4  0.991131         yo
6086      5  0.967857  yorkshire
6393      1  0.988955        zip

[1020 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 10, 2, 13, 20, 9, 6, 14, 8, 12, 15, 17, 19, 3, 7, 4, 18, 11, 16])